# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""13305658572674…",1.2927e19,"""74487505121589…","""../../../../da…","""17933777686893…","""17280186511082…",false
"""24939986540390…",1.4856e19,"""12299729890163…","""../../../../da…","""17933777686893…","""15056621451650…",true
"""32378938536206…",5.6458e17,"""82468457090785…","""../../../../da…","""11247036601166…","""17280186511082…",false
"""24939986540390…",4.0723e18,"""12351804747676…","""../../../../da…","""45405172147582…","""17280186511082…",false
"""13305658572674…",1.4677e19,"""62618498706752…","""../../../../da…","""45405172147582…","""65048035612336…",true
"""32378938536206…",1.6330e19,"""13078477532236…","""../../../../da…","""51583370832143…",null,false
"""32378938536206…",8.7675e18,"""87197509014624…","""../../../../da…","""45405172147582…","""65048035612336…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_MaritalStatus': 'PREDICTOR_0',
 'Customer_CLV': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""94850452336757…",1.2927e19,"""12467562131674…","""../../../../da…","""FirstMortgage3…","""14735036189694…","""Rejected"""
"""45066403561289…",1.4856e19,"""97733339689763…","""../../../../da…","""FirstMortgage3…","""88534907958160…","""Accepted"""
"""27049294717333…",5.6458e17,"""14277476629351…","""../../../../da…","""MoneyMarketSav…","""14735036189694…","""Rejected"""
"""45066403561289…",4.0723e18,"""51260424090835…","""../../../../da…","""BasicChecking""","""14735036189694…","""Rejected"""
"""94850452336757…",1.4677e19,"""15502134555710…","""../../../../da…","""BasicChecking""","""88504989034470…","""Accepted"""
"""27049294717333…",1.6330e19,"""48652275226140…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""27049294717333…",8.7675e18,"""12930675531379…","""../../../../da…","""BasicChecking""","""88504989034470…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""87358036244929…",1.2927e19,"""14482735904084…","""../../../../da…","""FirstMortgage3…","""14315117921970…","""Rejected"""
"""10277394125967…",1.4856e19,"""88019671542917…","""../../../../da…","""FirstMortgage3…","""66627677358512…","""Accepted"""
"""29559824582675…",5.6458e17,"""30534763861238…","""../../../../da…","""MoneyMarketSav…","""14315117921970…","""Rejected"""
"""10277394125967…",4.0723e18,"""13677079602633…","""../../../../da…","""BasicChecking""","""14315117921970…","""Rejected"""
"""87358036244929…",1.4677e19,"""13115225261914…","""../../../../da…","""BasicChecking""","""89202007915427…","""Accepted"""
"""29559824582675…",1.6330e19,"""17184955915242…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""29559824582675…",8.7675e18,"""42708070715967…","""../../../../da…","""BasicChecking""","""89202007915427…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_MaritalStatus=PREDICTOR_0
Customer_CLV=PREDICTOR_1
Customer_City=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""14214587975526…",1.2927e19,"""38743929781348…","""FirstMortgage3…","""55583091195571…","""Rejected"""
"""17706709479832…",1.4856e19,"""98433647867469…","""FirstMortgage3…","""51829006413901…","""Accepted"""
"""19621844161154…",5.6458e17,"""16418573901342…","""MoneyMarketSav…","""55583091195571…","""Rejected"""
"""17706709479832…",4.0723e18,"""15526209862534…","""BasicChecking""","""55583091195571…","""Rejected"""
"""14214587975526…",1.4677e19,"""14447707326953…","""BasicChecking""","""28089138431206…","""Accepted"""
"""19621844161154…",1.6330e19,"""84764540540044…","""UPlusFinPerson…",null,"""Rejected"""
"""19621844161154…",8.7675e18,"""18034619824325…","""BasicChecking""","""28089138431206…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'